In [1]:
import pandas as pd
import numpy as np

# Load the dataset
df = pd.read_csv('Mahindra_Employee_Reviews_from_AmbitionBox.csv')

# Audit Outputs
print("--- DF Shape ---")
print(df.shape)

print("\n--- DF Head ---")
print(df.head())

print("\n--- DF Info ---")
df.info()

print("\n--- Missing Values Audit ---")
print(df.isna().sum().sort_values(ascending=False))

print("\n--- Duplicate Rows Count ---")
print(f"Total Duplicates: {df.duplicated().sum()}")

--- DF Shape ---
(8614, 14)

--- DF Head ---
                           Title                  Place   Job_type  \
0                      Team Lead                   Pune     Intern   
1                    Back Office                 Mumbai  Full Time   
2                 Deputy Manager    Nashik, Maharashtra  Full Time   
3                         Intern                 Mumbai     Intern   
4  Assistant Manager Maintenance  Igatpuri, Maharashtra  Full Time   

                                     Department        Date  Overall_rating  \
0  Operations, Maintenance & Support Department  6 Sep 2023             2.0   
1                        Back Office Department  5 Sep 2023             3.0   
2  Operations, Maintenance & Support Department  4 Sep 2023             5.0   
3              Accounting & Taxation Department  2 Sep 2023             5.0   
4  Operations, Maintenance & Support Department  2 Sep 2023             5.0   

   work_life_balance  skill_development  salary_and_benefit

In [2]:
# Create a copy for cleaning
df_clean = df.copy()

# 1. Action: Remove duplicates (Additional Action 1)
df_clean = df_clean.drop_duplicates()

# 2. Action: Fix Datatype - Date to datetime (Mandatory Action 1)
df_clean['Date'] = pd.to_datetime(df_clean['Date'], errors='coerce')

# 3. Action: Handle Missing - Median for numeric (Mandatory Action 2a)
num_cols = df_clean.select_dtypes(include=[np.number]).columns
df_clean[num_cols] = df_clean[num_cols].fillna(df_clean[num_cols].median())

# 4. Action: Handle Missing - "Unknown" for text/categorical (Mandatory Action 2b)
cat_cols = df_clean.select_dtypes(include=['object']).columns
df_clean[cat_cols] = df_clean[cat_cols].fillna("Unknown")

# 5. Action: Clean text columns - Strip & Title case (Mandatory Action 3)
df_clean['Title'] = df_clean['Title'].str.strip().str.title()
df_clean['Place'] = df_clean['Place'].str.strip().str.title()
df_clean['Department'] = df_clean['Department'].str.strip().str.title()

# 6. Action: Rename columns to clean snake_case (Additional Action 2)
df_clean.columns = [col.lower().replace(" ", "_") for col in df_clean.columns]

# Show missing values before and after (Final Check)
print("\n--- Total Missing Values After Cleaning ---")
print(df_clean.isna().sum().sum()) 
# Note: remaining NaNs might exist in 'date' if parsing failed; we drop them:
df_clean = df_clean.dropna(subset=['date'])


--- Total Missing Values After Cleaning ---
1065


In [3]:
# 1. Extraction: Review Year from Date
df_clean['review_year'] = df_clean['date'].dt.year.astype(int)

# 2. Conditional Logic: Satisfaction Flag (Mandatory)
df_clean['is_high_satisfaction'] = df_clean['overall_rating'] >= 4.0

# 3. Mapping/Dictionary: Create a Region Category (Simplified)
def categorize_role(title):
    if 'Manager' in title or 'Lead' in title:
        return 'Management'
    elif 'Intern' in title or 'Trainee' in title:
        return 'Entry Level'
    else:
        return 'Professional'

df_clean['role_category'] = df_clean['title'].apply(categorize_role)

print("\n--- New Columns Preview ---")
print(df_clean[['review_year', 'is_high_satisfaction', 'role_category']].head())


--- New Columns Preview ---
   review_year  is_high_satisfaction role_category
0         2023                 False    Management
1         2023                 False  Professional
2         2023                  True    Management
3         2023                  True   Entry Level
4         2023                  True    Management


In [4]:
print("\n--- Generated Insights ---")

# 1. groupby().agg() - Avg ratings per role category
insight1 = df_clean.groupby('role_category')['overall_rating'].agg(['mean', 'count'])
print(f"Insight 1: The average rating for each role tier is:\n{insight1}")

# 2. sort_values() - Top 5 departments by work-life balance
insight2 = df_clean.groupby('department')['work_life_balance'].mean().sort_values(ascending=False).head(5)
print(f"\nInsight 2: The top 5 departments for Work-Life Balance are:\n{insight2}")

# 3. value_counts() - Distribution of Job Types
insight3 = df_clean['job_type'].value_counts()
print(f"\nInsight 3: The distribution of employment types is:\n{insight3}")

# 4. pivot_table() - Average career growth by year and role category
insight4 = df_clean.pivot_table(index='review_year', columns='role_category', values='career_growth', aggfunc='mean')
print(f"\nInsight 4: Career growth trends over the years:\n{insight4.tail()}")

# 5. Filtering - Highly satisfied employees in Management
insight5_count = df_clean[(df_clean['role_category'] == 'Management') & (df_clean['overall_rating'] == 5.0)].shape[0]
print(f"\nInsight 5: There are {insight5_count} managers who gave a perfect 5.0 overall rating.")


--- Generated Insights ---
Insight 1: The average rating for each role tier is:
                   mean  count
role_category                 
Entry Level    4.335097    567
Management     4.150000   2820
Professional   4.203153   3933

Insight 2: The top 5 departments for Work-Life Balance are:
department
45Jph Paintshop G Block Department    5.0
टिंकू  Department                     5.0
Zirakpur Department                   5.0
Zaheerabd Department                  5.0
80% Department                        5.0
Name: work_life_balance, dtype: float64

Insight 3: The distribution of employment types is:
job_type
Unknown        4758
Full Time      2418
Contractual      60
Intern           53
Part Time        28
Freelancer        3
Name: count, dtype: int64

Insight 4: Career growth trends over the years:
role_category  Entry Level  Management  Professional
review_year                                         
2019              4.145455    4.002695      4.101597
2020              4.275362

In [5]:
topath = r"C:\Users\Kiran S\OneDrive\Desktop\vishwas assignment 2.csv"
df.to_csv(topath,index=False)